In [ ]:
%module load ffmpeg-XXX

In [ ]:
!git clone https://github.com/cyrta/dscaper
!pip install -e ./dscaper

In [ ]:
# On MacOS
!brew install ffmpeg
!brew install sox

In [ ]:
!CFLAGS="-I/Users/yanislabrak/opt/miniconda3/envs/sdialog/include" LDFLAGS="-L/Users/yanislabrak/opt/miniconda3/envs/sdialog/lib" pip install -e dscaper --force-reinstall --no-cache-dir

In [ ]:
import scaper
from scaper.dscaper_datatypes import DscaperAudio, DscaperTimeline, DscaperBackground, DscaperEvent, DscaperGenerate

In [ ]:
DATA_PATH = "./dcaper_data"
dsc = scaper.Dscaper(dscaper_base_path=DATA_PATH)

In [ ]:
utterances = ["./outputs/2first_dialog_audio_elevenlabs.wav" for _ in range(5)]

for utt in utterances:
    metadata = DscaperAudio(library="dialog_1", label="speaker_A", filename="turn_1.wav")
    resp = dsc.store_audio(utt, metadata)
    if resp.status != "success":
        print("Problem!")

In [ ]:
audio_events = [[
    "./outputs/keyboard_typing.wav",
    "room_objects",
    "keyboard_typing",
    f"keyboard_typing_{i}.wav"
] for i in range(5)]

for audio_event in audio_events:
    metadata = DscaperAudio(library=audio_event[1], label=audio_event[2], filename=audio_event[3])
    resp = dsc.store_audio(audio_event[0], metadata)
    if resp.status != "success":
        print("Problem!")

In [ ]:
audio_events = [[
    "./outputs/ac_noise.wav",
    "background",
    "ac_noise",
    f"ac_noise_{i}.wav"
] for i in range(2)]

for audio_event in audio_events:
    metadata = DscaperAudio(library=audio_event[1], label=audio_event[2], filename=audio_event[3])
    resp = dsc.store_audio(audio_event[0], metadata)
    if resp.status != "success":
        print("Problem!")

# Build the timeline

In [ ]:
timeline_metadata = DscaperTimeline(name="dialog_1", duration=60.0, description="Test timeline") # WARNING: time
dsc.create_timeline(timeline_metadata)

## Background are repeated

In [ ]:
background_metadata = DscaperBackground(
    library="background",
    label=["const", "ac_noise"],
    source_file=["choose","[]"]
)
dsc.add_background("dialog_1", background_metadata) # Repeat it indefinitely

## Event are fixed elements

In [ ]:
event_metadata = DscaperEvent(
    library="room_objects",
    label=["const", "keyboard_typing"],
    source_file=["choose","[]"],
    event_time=["const","20"], # 20s after the start of the timeline
    event_duration=None, # ["const","1"] 1s duration
    position="seat_1", # need to be generated at the same time as the SNR
    speaker=None, # speaker_1 if utterance
    text=None, # "Hello, how are you?" if utterance
)
dsc.add_event("dialog_1", event_metadata)

In [ ]:
utt_metadata = DscaperEvent(
    library="room_objects",
    label=["const", "keyboard_typing"],
    source_file=["choose","[]"],
    event_time=["const","20"], # 20s after the start of the timeline
    event_duration=None, # ["const","1"] 1s duration
    position="door", # need to be generated at the same time as the SNR
    speaker="speaker_1",
    text="Hello how are you",
)
dsc.add_event("dialog_1", utt_metadata)

In [ ]:
generate_metadata = DscaperGenerate(seed=0, save_isolated_positions=True)
resp = dsc.generate_timeline("dialog_1", generate_metadata)

content = DscaperGenerate(**resp.content)
print("ID:",content.id)
# Pavel need: DATA_PATH > generate > id

# For the timeline annotation viz

In [ ]:
import misc.jams_to_rttm as jams2rttm
import misc.jams_to_textgrid as jams2textgrid

In [ ]:
jams_path = "/path/to/input.jams"
rttm_path = "/path/to/output.rttm"
jams2rttm.jams_to_rttm(jams_path, rttm_path)

In [ ]:
jams_path = "/path/to/input.jams"
textgrid_path = "/path/to/output.textgrid"
jams2textgrid.jams_to_textgrid(jams_path, textgrid_path)